# CCC1 - Combined Collaborative Approach (Two-Stage Filtering)

## Strategy
- **Stage 1 (CCA2)**: Filter candidates with high connection probability
  - Use CCA2 model to score all items
  - Select Top-K candidates per user
- **Stage 2 (CCB2)**: Predict rating and filter by quality
  - Use CCB2 model to predict rating for candidates
  - Recommend if predicted_rating >= threshold
- **Combine**: Only items passing both stages are recommended

## Key Features:
- ✅ Leverages both CCA2 (connection) and CCB2 (rating) models
- ✅ No additional training needed (uses pretrained models)
- ✅ Highly interpretable: clear two-stage decision process
- ✅ Flexible: adjust CCA_TOP_K and CCB_THRESHOLD independently

## Hyperparameters:
- `CCA_TOP_K_CANDIDATES`: 100 (number of candidates from Stage 1)
- `CCB_RATING_THRESHOLD`: 4.0 (minimum rating for recommendation)
- `GOOD_RATING_THRESHOLD`: 4.0 (ground truth definition)

## Expected Performance:
- Better than CCA2: Stage 2 filters low-quality connections
- Better than CCB2: Stage 1 focuses on realistic candidates
- AUC-ROC target: 0.93+
- F1 target: 0.87+

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from collections import defaultdict
import warnings
warnings.filterwarnings('ignore')
import time

import torch
import torch.nn as nn
from sklearn.metrics import roc_auc_score, precision_score, recall_score, f1_score

SEED = 42
np.random.seed(SEED)
torch.manual_seed(SEED)

# Device setup (CUDA > MPS > CPU)
if torch.cuda.is_available():
    device = torch.device('cuda')
    print(f'Device: {device} ({torch.cuda.get_device_name()})')
elif torch.backends.mps.is_available():
    device = torch.device('mps')
    print(f'Device: {device}')
else:
    device = torch.device('cpu')
    print(f'Device: {device}')

Device: mps


## 1. Data Preprocessing

### CCC1 Strategy:
- Uses same data preprocessing as CCB (rating-based split)
- Positive: Rating >= 4 (for evaluation)
- Train on all interactions (for graph structure)

In [2]:
df = pd.read_csv('../data/train.csv')

print(f"Total interactions: {len(df):,}")
print(f"Unique users: {df['user'].nunique()}")
print(f"Unique items: {df['item'].nunique()}")
print(f"\nRating distribution:")
print(df['rating'].value_counts().sort_index())

# Define good purchases
GOOD_RATING_THRESHOLD = 4.0
n_good_purchases = (df['rating'] >= GOOD_RATING_THRESHOLD).sum()
print(f"\nGood purchases (rating >= {GOOD_RATING_THRESHOLD}): {n_good_purchases:,} ({100*n_good_purchases/len(df):.1f}%)")

# ID mapping
user2idx = {u: i for i, u in enumerate(sorted(df['user'].unique()))}
item2idx = {it: i for i, it in enumerate(sorted(df['item'].unique()))}
idx2user = {i: u for u, i in user2idx.items()}
idx2item = {i: it for it, i in item2idx.items()}

n_users, n_items = len(user2idx), len(item2idx)

df['user_idx'] = df['user'].map(user2idx)
df['item_idx'] = df['item'].map(item2idx)

print(f"\nUsers: {n_users}, Items: {n_items}")
print(f"Sparsity: {(1 - len(df) / (n_users * n_items)) * 100:.4f}%")

Total interactions: 105,139
Unique users: 668
Unique items: 10321

Rating distribution:
rating
0.5     1189
1.0     3254
1.5     1564
2.0     7929
2.5     5473
3.0    21676
3.5    12224
4.0    28831
4.5     8174
5.0    14825
Name: count, dtype: int64

Good purchases (rating >= 4.0): 51,830 (49.3%)

Users: 668, Items: 10321
Sparsity: 98.4750%


In [3]:
# User별 interaction count 및 K값 계산
user_interaction_count = df.groupby('user_idx').size().to_dict()

MAX_K = 100

def get_k_for_user(count):
    """User별 추천 개수 K 계산 (평가 규칙: 20% 이하 추천)"""
    if count <= 10:
        return 2
    k = max(2, int(count * 0.2))
    return min(k, MAX_K)

user_k = {u: get_k_for_user(c) for u, c in user_interaction_count.items()}

k_values = list(user_k.values())
print(f"User K values statistics (MAX_K={MAX_K}):")
print(f"  Min K: {min(k_values)}")
print(f"  Max K: {max(k_values)}")
print(f"  Mean K: {np.mean(k_values):.2f}")
print(f"  Median K: {np.median(k_values):.2f}")

User K values statistics (MAX_K=100):
  Min K: 4
  Max K: 100
  Mean K: 24.74
  Median K: 14.00


## 2. Train/Val/Test Split

### CCC1 Split (CCB-style):
- **Good purchases (rating >= 4)**: Split into train/val/test (7:1.5:1.5)
- **Poor purchases (rating < 4)**: All go to train
- **Evaluation**: Only on rating >= 4

In [4]:
# Train/Val/Test Split (70/15/15) - Rating-aware
train_data, val_data, test_data = [], [], []

for user_idx in range(n_users):
    user_df = df[df['user_idx'] == user_idx]
    
    good_purchases = user_df[user_df['rating'] >= GOOD_RATING_THRESHOLD][['user_idx', 'item_idx', 'rating']]
    bad_purchases = user_df[user_df['rating'] < GOOD_RATING_THRESHOLD][['user_idx', 'item_idx', 'rating']]
    
    if len(bad_purchases) > 0:
        train_data.append(bad_purchases[['user_idx', 'item_idx']])
    
    n_good = len(good_purchases)
    
    if n_good >= 3:
        good_purchases = good_purchases.sample(frac=1, random_state=SEED).reset_index(drop=True)
        train_end = int(0.7 * n_good)
        val_end = train_end + int(0.15 * n_good)
        
        train_end = max(1, train_end)
        val_end = max(train_end + 1, val_end)
        
        train_data.append(good_purchases.iloc[:train_end][['user_idx', 'item_idx']])
        val_data.append(good_purchases.iloc[train_end:val_end][['user_idx', 'item_idx']])
        test_data.append(good_purchases.iloc[val_end:][['user_idx', 'item_idx']])
    elif n_good == 2:
        good_purchases = good_purchases.sample(frac=1, random_state=SEED).reset_index(drop=True)
        train_data.append(good_purchases.iloc[:1][['user_idx', 'item_idx']])
        val_data.append(good_purchases.iloc[1:][['user_idx', 'item_idx']])
    elif n_good == 1:
        train_data.append(good_purchases[['user_idx', 'item_idx']])

train_df = pd.concat(train_data, ignore_index=True)
val_df = pd.concat(val_data, ignore_index=True) if val_data else pd.DataFrame(columns=['user_idx', 'item_idx'])
test_df = pd.concat(test_data, ignore_index=True) if test_data else pd.DataFrame(columns=['user_idx', 'item_idx'])

print(f"Train edges (all purchases for graph): {len(train_df):,}")
print(f"Val edges (good purchases only): {len(val_df):,}")
print(f"Test edges (good purchases only): {len(test_df):,}")

n_val_test = len(val_df) + len(test_df)
print(f"\nGood purchases in Val/Test: {n_val_test:,} / {n_good_purchases:,} ({100*n_val_test/n_good_purchases:.1f}%)")

Train edges (all purchases for graph): 89,294
Val edges (good purchases only): 7,480
Test edges (good purchases only): 8,365

Good purchases in Val/Test: 15,845 / 51,830 (30.6%)


In [5]:
# User가 train에서 선택한 items (추천 시 제외용)
user_train_items = defaultdict(set)
for u, i in zip(train_df['user_idx'].values, train_df['item_idx'].values):
    user_train_items[int(u)].add(int(i))

print(f"User train items dictionary ready: {len(user_train_items)} users")

User train items dictionary ready: 668 users


## 3. Load Pretrained Models

### Load CCA2 and CCB2:
- **CCA2**: For connection probability (Stage 1)
- **CCB2**: For rating prediction (Stage 2)
- Both models use same graph structure

In [6]:
# Model definitions (copy from CCA2 and CCB2)

class LightGCN(nn.Module):
    """LightGCN for CCA2 (connection prediction)"""
    def __init__(self, n_users, n_items, emb_dim=64, n_layers=2):
        super().__init__()
        self.n_users = n_users
        self.n_items = n_items
        self.emb_dim = emb_dim
        self.n_layers = n_layers
        
        self.user_emb = nn.Embedding(n_users, emb_dim)
        self.item_emb = nn.Embedding(n_items, emb_dim)
        nn.init.xavier_uniform_(self.user_emb.weight)
        nn.init.xavier_uniform_(self.item_emb.weight)
    
    def forward(self, edge_index, edge_weight):
        all_emb = torch.cat([self.user_emb.weight, self.item_emb.weight], dim=0)
        embs = [all_emb]
        
        for _ in range(self.n_layers):
            row, col = edge_index
            messages = all_emb[col] * edge_weight.unsqueeze(1)
            all_emb = torch.zeros_like(all_emb).scatter_add(
                0, row.unsqueeze(1).expand(-1, self.emb_dim), messages
            )
            embs.append(all_emb)
        
        final_emb = torch.mean(torch.stack(embs), dim=0)
        return final_emb[:self.n_users], final_emb[self.n_users:]


class LightGCN_with_Rating(nn.Module):
    """LightGCN with Rating Head for CCB2 (rating prediction)"""
    def __init__(self, n_users, n_items, emb_dim=64, n_layers=2):
        super().__init__()
        self.n_users = n_users
        self.n_items = n_items
        self.emb_dim = emb_dim
        self.n_layers = n_layers
        
        self.user_emb = nn.Embedding(n_users, emb_dim)
        self.item_emb = nn.Embedding(n_items, emb_dim)
        nn.init.xavier_uniform_(self.user_emb.weight)
        nn.init.xavier_uniform_(self.item_emb.weight)
        
        self.rating_mlp = nn.Sequential(
            nn.Linear(emb_dim, 32),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(32, 1)
        )
    
    def forward(self, edge_index, edge_weight):
        all_emb = torch.cat([self.user_emb.weight, self.item_emb.weight], dim=0)
        embs = [all_emb]
        
        for _ in range(self.n_layers):
            row, col = edge_index
            messages = all_emb[col] * edge_weight.unsqueeze(1)
            all_emb = torch.zeros_like(all_emb).scatter_add(
                0, row.unsqueeze(1).expand(-1, self.emb_dim), messages
            )
            embs.append(all_emb)
        
        final_emb = torch.mean(torch.stack(embs), dim=0)
        return final_emb[:self.n_users], final_emb[self.n_users:]
    
    def predict_rating(self, user_idx, item_idx, edge_index, edge_weight):
        u_emb, i_emb = self.forward(edge_index, edge_weight)
        interaction = u_emb[user_idx] * i_emb[item_idx]
        rating_logit = self.rating_mlp(interaction).squeeze(-1)
        predicted_rating = torch.sigmoid(rating_logit) * 4.5 + 0.5
        return predicted_rating


print("Model classes defined")

Model classes defined


In [7]:
# Build graphs for both models

def build_unweighted_graph():
    """Unweighted graph for CCA2"""
    users = train_df['user_idx'].values
    items = train_df['item_idx'].values
    
    edge_u2i = np.array([users, items + n_users])
    edge_i2u = np.array([items + n_users, users])
    edge_index = torch.LongTensor(np.concatenate([edge_u2i, edge_i2u], axis=1))
    
    num_nodes = n_users + n_items
    deg = torch.zeros(num_nodes).scatter_add(0, edge_index[0], torch.ones(edge_index.shape[1]))
    deg_inv_sqrt = deg.pow(-0.5)
    deg_inv_sqrt[deg_inv_sqrt == float('inf')] = 0
    
    edge_weight = deg_inv_sqrt[edge_index[0]] * deg_inv_sqrt[edge_index[1]]
    
    return edge_index.to(device), edge_weight.to(device)


def build_rating_weighted_graph():
    """Rating weighted graph for CCB2"""
    users = train_df['user_idx'].values
    items = train_df['item_idx'].values
    
    ratings = []
    for u, i in zip(users, items):
        rating = df[(df['user_idx'] == u) & (df['item_idx'] == i)]['rating'].values
        ratings.append(rating[0] if len(rating) > 0 else 3)
    ratings = np.array(ratings)
    
    rating_factors = 0.4 + 0.15 * ratings
    
    edge_u2i = np.array([users, items + n_users])
    edge_i2u = np.array([items + n_users, users])
    edge_index = torch.LongTensor(np.concatenate([edge_u2i, edge_i2u], axis=1))
    
    rating_factors_both = np.concatenate([rating_factors, rating_factors])
    
    num_nodes = n_users + n_items
    deg = torch.zeros(num_nodes).scatter_add(0, edge_index[0], torch.ones(edge_index.shape[1]))
    deg_inv_sqrt = deg.pow(-0.5)
    deg_inv_sqrt[deg_inv_sqrt == float('inf')] = 0
    
    base_weight = deg_inv_sqrt[edge_index[0]] * deg_inv_sqrt[edge_index[1]]
    rating_weight = torch.FloatTensor(rating_factors_both)
    edge_weight = base_weight * rating_weight
    
    return edge_index.to(device), edge_weight.to(device)


# Build both graphs
cca_edge_index, cca_edge_weight = build_unweighted_graph()
ccb_edge_index, ccb_edge_weight = build_rating_weighted_graph()

print(f"CCA Graph (unweighted): {cca_edge_index.shape[1]:,} edges")
print(f"CCB Graph (rating-weighted): {ccb_edge_index.shape[1]:,} edges")

CCA Graph (unweighted): 178,588 edges
CCB Graph (rating-weighted): 178,588 edges


In [8]:
# Load pretrained models

EMB_DIM = 32
N_LAYERS = 2

# CCA2 model
cca_model = LightGCN(n_users, n_items, EMB_DIM, N_LAYERS).to(device)
cca_model.load_state_dict(torch.load('../cc_models/cca2_best.pt'))
cca_model.eval()
print(f"✓ CCA2 model loaded from ../cc_models/cca2_best.pt")

# CCB2 model
ccb_model = LightGCN_with_Rating(n_users, n_items, EMB_DIM, N_LAYERS).to(device)
ccb_model.load_state_dict(torch.load('../cc_models/ccb2_best.pt'))
ccb_model.eval()
print(f"✓ CCB2 model loaded from ../cc_models/ccb2_best.pt")

print(f"\nBoth models ready for Two-Stage inference!")

✓ CCA2 model loaded from ../cc_models/cca2_best.pt
✓ CCB2 model loaded from ../cc_models/ccb2_best.pt

Both models ready for Two-Stage inference!


## 4. Two-Stage Recommendation

### Implementation:
```
Stage 1 (CCA2):
  - Score all candidate items with CCA model
  - Select Top-K candidates (CCA_TOP_K_CANDIDATES)

Stage 2 (CCB2):
  - Predict rating for each candidate with CCB model
  - Recommend if predicted_rating >= CCB_RATING_THRESHOLD
```

In [9]:
# Hyperparameters for Two-Stage
CCA_TOP_K_CANDIDATES = 100  # Stage 1: Top-K from CCA
CCB_RATING_THRESHOLD = 4.0  # Stage 2: Rating threshold for CCB

print(f"Two-Stage Hyperparameters:")
print(f"  Stage 1 (CCA): Top-{CCA_TOP_K_CANDIDATES} candidates")
print(f"  Stage 2 (CCB): Rating >= {CCB_RATING_THRESHOLD}")

Two-Stage Hyperparameters:
  Stage 1 (CCA): Top-100 candidates
  Stage 2 (CCB): Rating >= 4.0


In [10]:
def predict_two_stage(test_input_df, cca_top_k=100, ccb_threshold=4.0, verbose=True, show_details=False):
    """
    ★ CCC1: Two-Stage Recommendation
    
    Stage 1 (CCA): Filter candidates by connection probability
    Stage 2 (CCB): Filter by predicted rating quality
    
    Args:
        test_input_df: Test data (user, item columns)
        cca_top_k: Number of candidates from Stage 1
        ccb_threshold: Rating threshold for Stage 2
        verbose: Print AGENTS.md format
        show_details: Show Stage 1/2 decisions
    
    Returns:
        results_df: DataFrame with recommendations
    """
    cca_model.eval()
    ccb_model.eval()
    
    # Get embeddings
    with torch.no_grad():
        cca_u_emb, cca_i_emb = cca_model(cca_edge_index, cca_edge_weight)
    
    results = []
    stats = {
        'total_o': 0, 
        'total_items': 0,
        'stage1_pass': 0,
        'stage2_pass': 0
    }

    for user in test_input_df['user'].unique():
        if user not in user2idx:
            # Unknown user: all X
            user_rows = test_input_df[test_input_df['user'] == user]
            for _, row in user_rows.iterrows():
                results.append({
                    'user': row['user'], 
                    'item': row['item'], 
                    'recommend': 'X',
                    'stage': 'unknown_user'
                })
                stats['total_items'] += 1
            continue

        user_idx = user2idx[user]
        user_rows = test_input_df[test_input_df['user'] == user]
        train_items_set = user_train_items[user_idx]

        # Stage 1: CCA - Get Top-K candidates
        candidate_items = [i for i in range(n_items) if i not in train_items_set]
        
        if len(candidate_items) == 0:
            for _, row in user_rows.iterrows():
                results.append({
                    'user': user,
                    'item': row['item'],
                    'recommend': 'X',
                    'stage': 'no_candidates'
                })
                stats['total_items'] += 1
            continue
        
        # Score all candidates with CCA
        candidate_tensor = torch.LongTensor(candidate_items).to(device)
        user_tensor = torch.full((len(candidate_items),), user_idx, dtype=torch.long, device=device)
        
        with torch.no_grad():
            cca_scores = (cca_u_emb[user_tensor] * cca_i_emb[candidate_tensor]).sum(dim=1).cpu().numpy()
        
        # Select Top-K from CCA
        K = min(cca_top_k, len(candidate_items))
        top_k_indices = np.argsort(cca_scores)[-K:]
        stage1_candidates = {candidate_items[idx]: cca_scores[idx] for idx in top_k_indices}
        
        # Process test items
        for _, row in user_rows.iterrows():
            item = row['item']
            stats['total_items'] += 1
            
            # Check unknown item or train item
            if item not in item2idx:
                results.append({
                    'user': user,
                    'item': item,
                    'recommend': 'X',
                    'stage': 'unknown_item'
                })
                continue
            
            item_idx = item2idx[item]
            
            if item_idx in train_items_set:
                results.append({
                    'user': user,
                    'item': item,
                    'recommend': 'X',
                    'stage': 'in_train'
                })
                continue
            
            # Stage 1: Check if in Top-K from CCA
            if item_idx not in stage1_candidates:
                results.append({
                    'user': user,
                    'item': item,
                    'recommend': 'X',
                    'stage': 'stage1_filtered'
                })
                continue
            
            stats['stage1_pass'] += 1
            
            # Stage 2: CCB - Predict rating
            user_tensor = torch.tensor([user_idx], dtype=torch.long).to(device)
            item_tensor = torch.tensor([item_idx], dtype=torch.long).to(device)
            
            with torch.no_grad():
                pred_rating = ccb_model.predict_rating(
                    user_tensor, item_tensor,
                    ccb_edge_index, ccb_edge_weight
                ).item()
            
            # Stage 2: Check rating threshold
            if pred_rating >= ccb_threshold:
                recommend = 'O'
                stats['total_o'] += 1
                stats['stage2_pass'] += 1
                stage = 'both_pass'
            else:
                recommend = 'X'
                stage = 'stage2_filtered'
            
            results.append({
                'user': user,
                'item': item,
                'recommend': recommend,
                'stage': stage,
                'cca_score': stage1_candidates[item_idx],
                'ccb_rating': pred_rating
            })

    results_df = pd.DataFrame(results)
    
    # Print results
    if verbose:
        print("=" * 70)
        if show_details:
            print(f"{'user':<10} {'item':<10} {'CCA_score':<12} {'CCB_rating':<12} {'recommend':<10} {'stage':<15}")
            for _, row in results_df.iterrows():
                cca_s = f"{row.get('cca_score', 0):.4f}" if 'cca_score' in row else 'N/A'
                ccb_r = f"{row.get('ccb_rating', 0):.2f}" if 'ccb_rating' in row else 'N/A'
                print(f"{row['user']:<10} {row['item']:<10} {cca_s:<12} {ccb_r:<12} {row['recommend']:<10} {row['stage']:<15}")
        else:
            print(f"{'user':<10} {'item':<10} {'recommend':<10}")
            for _, row in results_df.iterrows():
                print(f"{row['user']:<10} {row['item']:<10} {row['recommend']:<10}")
        print("=" * 70)
        print(f"Total recommends = {stats['total_o']}/{stats['total_items']}")
        print(f"Not recommend = {stats['total_items'] - stats['total_o']}/{stats['total_items']}")
        print(f"\nTwo-Stage Statistics:")
        print(f"  Stage 1 pass rate: {stats['stage1_pass']}/{stats['total_items']} ({100*stats['stage1_pass']/stats['total_items']:.1f}%)")
        print(f"  Stage 2 pass rate: {stats['stage2_pass']}/{stats['stage1_pass']} ({100*stats['stage2_pass']/stats['stage1_pass']:.1f}% of Stage 1)")
        print()

    return results_df


print("Two-Stage prediction function ready!")

Two-Stage prediction function ready!


## 5. Sample Prediction Test

In [11]:
# Test with sample1.csv
sample1 = pd.read_csv('../data/sample1.csv')

print("Sample1.csv Test (CCC1 - Two-Stage):")
print(f"Stage 1 (CCA): Top-{CCA_TOP_K_CANDIDATES} candidates")
print(f"Stage 2 (CCB): Rating >= {CCB_RATING_THRESHOLD}")
print()
print("Predictions with details:")
predictions1 = predict_two_stage(sample1, CCA_TOP_K_CANDIDATES, CCB_RATING_THRESHOLD, verbose=True, show_details=True)

Sample1.csv Test (CCC1 - Two-Stage):
Stage 1 (CCA): Top-100 candidates
Stage 2 (CCB): Rating >= 4.0

Predictions with details:
user       item       CCA_score    CCB_rating   recommend  stage          
109        3745       nan          nan          X          stage1_filtered
88         4447       nan          nan          X          stage1_filtered
71         4306       1.8564       4.73         O          both_pass      
66         1747       1.6747       3.45         X          stage2_filtered
15         66934      nan          nan          X          stage1_filtered
Total recommends = 1/5
Not recommend = 4/5

Two-Stage Statistics:
  Stage 1 pass rate: 2/5 (40.0%)
  Stage 2 pass rate: 1/2 (50.0% of Stage 1)



In [12]:
# Test with sample2.csv
sample2 = pd.read_csv('../data/sample2.csv')

print("Sample2.csv Test (CCC1 - Two-Stage):")
print(f"Stage 1 (CCA): Top-{CCA_TOP_K_CANDIDATES} candidates")
print(f"Stage 2 (CCB): Rating >= {CCB_RATING_THRESHOLD}")
print()
print("Predictions with details:")
predictions2 = predict_two_stage(sample2, CCA_TOP_K_CANDIDATES, CCB_RATING_THRESHOLD, verbose=True, show_details=True)

Sample2.csv Test (CCC1 - Two-Stage):
Stage 1 (CCA): Top-100 candidates
Stage 2 (CCB): Rating >= 4.0

Predictions with details:
user       item       CCA_score    CCB_rating   recommend  stage          
109        3745.0     nan          nan          X          stage1_filtered
88         4447.0     nan          nan          X          stage1_filtered
71         4306.0     1.8564       4.73         O          both_pass      
66         1747.0     1.6747       3.45         X          stage2_filtered
15         66934.0    nan          nan          X          stage1_filtered
Total recommends = 1/5
Not recommend = 4/5

Two-Stage Statistics:
  Stage 1 pass rate: 2/5 (40.0%)
  Stage 2 pass rate: 1/2 (50.0% of Stage 1)



## 6. Validation Evaluation

In [13]:
# Convert val_df to test format
val_test_df = val_df.copy()
val_test_df['user'] = val_test_df['user_idx'].map(idx2user)
val_test_df['item'] = val_test_df['item_idx'].map(idx2item)
val_test_df = val_test_df[['user', 'item']]

print(f"Evaluating on validation set: {len(val_test_df)} samples")
print()

val_predictions = predict_two_stage(val_test_df, CCA_TOP_K_CANDIDATES, CCB_RATING_THRESHOLD, verbose=False)

# All items in val are positive (rating >= 4)
val_labels = np.ones(len(val_predictions))
val_preds = (val_predictions['recommend'] == 'O').astype(int).values

val_acc = (val_preds == val_labels).mean()
val_prec = precision_score(val_labels, val_preds, zero_division=0)
val_rec = recall_score(val_labels, val_preds, zero_division=0)
val_f1 = f1_score(val_labels, val_preds, zero_division=0)

print(f"\nValidation Performance (CCC1 - Two-Stage):")
print(f"  Accuracy: {val_acc:.4f}")
print(f"  Precision: {val_prec:.4f}")
print(f"  Recall: {val_rec:.4f}")
print(f"  F1 Score: {val_f1:.4f}")
print(f"  O ratio: {val_preds.mean()*100:.1f}%")

Evaluating on validation set: 7480 samples


Validation Performance (CCC1 - Two-Stage):
  Accuracy: 0.2136
  Precision: 1.0000
  Recall: 0.2136
  F1 Score: 0.3521
  O ratio: 21.4%


## 7. Test Evaluation

In [14]:
# Convert test_df to test format
test_test_df = test_df.copy()
test_test_df['user'] = test_test_df['user_idx'].map(idx2user)
test_test_df['item'] = test_test_df['item_idx'].map(idx2item)
test_test_df = test_test_df[['user', 'item']]

print(f"Evaluating on test set: {len(test_test_df)} samples")
print()

test_predictions = predict_two_stage(test_test_df, CCA_TOP_K_CANDIDATES, CCB_RATING_THRESHOLD, verbose=False)

# All items in test are positive (rating >= 4)
test_labels = np.ones(len(test_predictions))
test_preds = (test_predictions['recommend'] == 'O').astype(int).values

test_acc = (test_preds == test_labels).mean()
test_prec = precision_score(test_labels, test_preds, zero_division=0)
test_rec = recall_score(test_labels, test_preds, zero_division=0)
test_f1 = f1_score(test_labels, test_preds, zero_division=0)

print(f"\nTest Performance (CCC1 - Two-Stage):")
print(f"  Accuracy: {test_acc:.4f}")
print(f"  Precision: {test_prec:.4f}")
print(f"  Recall: {test_rec:.4f}")
print(f"  F1 Score: {test_f1:.4f}")
print(f"  O ratio: {test_preds.mean()*100:.1f}%")

Evaluating on test set: 8365 samples


Test Performance (CCC1 - Two-Stage):
  Accuracy: 0.2245
  Precision: 1.0000
  Recall: 0.2245
  F1 Score: 0.3667
  O ratio: 22.5%


## 8. Full AUC-ROC Evaluation

To calculate AUC-ROC properly, we need positive and negative samples:

In [15]:
print("Calculating AUC-ROC with negative samples...")

# Positive samples: validation good purchases
val_pos_users = val_df['user_idx'].values
val_pos_items = val_df['item_idx'].values

# Negative samples: random non-interactions
val_test_edges = set()
for u, i in zip(val_df['user_idx'].values, val_df['item_idx'].values):
    val_test_edges.add((int(u), int(i)))
for u, i in zip(test_df['user_idx'].values, test_df['item_idx'].values):
    val_test_edges.add((int(u), int(i)))

n_neg = len(val_df)
neg_users, neg_items = [], []
attempts = 0
max_attempts = n_neg * 100

while len(neg_users) < n_neg and attempts < max_attempts:
    u = np.random.randint(0, n_users)
    i = np.random.randint(0, n_items)
    attempts += 1
    
    if i not in user_train_items[u] and (u, i) not in val_test_edges:
        neg_users.append(u)
        neg_items.append(i)

# Score positive samples
pos_scores = []
cca_u_emb, cca_i_emb = cca_model(cca_edge_index, cca_edge_weight)

for u_idx, i_idx in zip(val_pos_users, val_pos_items):
    # Stage 1: CCA score
    with torch.no_grad():
        cca_score = (cca_u_emb[u_idx] * cca_i_emb[i_idx]).sum().item()
    
    # Stage 2: CCB rating (combine as final score)
    with torch.no_grad():
        u_t = torch.tensor([u_idx], dtype=torch.long).to(device)
        i_t = torch.tensor([i_idx], dtype=torch.long).to(device)
        ccb_rating = ccb_model.predict_rating(u_t, i_t, ccb_edge_index, ccb_edge_weight).item()
    
    # Combined score (normalized)
    combined_score = 0.5 * cca_score + 0.5 * (ccb_rating / 5.0)
    pos_scores.append(combined_score)

# Score negative samples
neg_scores = []
for u_idx, i_idx in zip(neg_users, neg_items):
    with torch.no_grad():
        cca_score = (cca_u_emb[u_idx] * cca_i_emb[i_idx]).sum().item()
        u_t = torch.tensor([u_idx], dtype=torch.long).to(device)
        i_t = torch.tensor([i_idx], dtype=torch.long).to(device)
        ccb_rating = ccb_model.predict_rating(u_t, i_t, ccb_edge_index, ccb_edge_weight).item()
    
    combined_score = 0.5 * cca_score + 0.5 * (ccb_rating / 5.0)
    neg_scores.append(combined_score)

# Calculate AUC-ROC
all_scores = np.concatenate([pos_scores, neg_scores])
all_labels = np.concatenate([np.ones(len(pos_scores)), np.zeros(len(neg_scores))])

val_auc = roc_auc_score(all_labels, all_scores)

print(f"\nValidation AUC-ROC: {val_auc:.4f}")
print(f"  Positive scores: mean={np.mean(pos_scores):.4f}, std={np.std(pos_scores):.4f}")
print(f"  Negative scores: mean={np.mean(neg_scores):.4f}, std={np.std(neg_scores):.4f}")

Calculating AUC-ROC with negative samples...

Validation AUC-ROC: 0.9562
  Positive scores: mean=1.0583, std=0.3497
  Negative scores: mean=0.2677, std=0.2316


## 9. Final Summary

In [16]:
print("="*70)
print("CCC1 - Combined Collaborative Approach (Two-Stage Filtering)")
print("="*70)

print(f"\nStrategy:")
print(f"  Stage 1 (CCA2): Connection probability filter")
print(f"    → Top-{CCA_TOP_K_CANDIDATES} candidates per user")
print(f"  Stage 2 (CCB2): Rating quality filter")
print(f"    → Recommend if predicted_rating >= {CCB_RATING_THRESHOLD}")

print(f"\nModels:")
print(f"  CCA2: {sum(p.numel() for p in cca_model.parameters()):,} parameters")
print(f"  CCB2: {sum(p.numel() for p in ccb_model.parameters()):,} parameters")

print(f"\nData:")
print(f"  Good purchases (rating >= {GOOD_RATING_THRESHOLD}): {n_good_purchases:,}")
print(f"  Train: {len(train_df):,}")
print(f"  Val: {len(val_df):,}")
print(f"  Test: {len(test_df):,}")

print(f"\nValidation Performance:")
print(f"  AUC-ROC: {val_auc:.4f}")
print(f"  Accuracy: {val_acc:.4f}")
print(f"  Precision: {val_prec:.4f}")
print(f"  Recall: {val_rec:.4f}")
print(f"  F1 Score: {val_f1:.4f}")

print(f"\nTest Performance:")
print(f"  Accuracy: {test_acc:.4f}")
print(f"  Precision: {test_prec:.4f}")
print(f"  Recall: {test_rec:.4f}")
print(f"  F1 Score: {test_f1:.4f}")

print(f"\nReady for comparison with CCA2, CCB2, and other CCC variants!")

CCC1 - Combined Collaborative Approach (Two-Stage Filtering)

Strategy:
  Stage 1 (CCA2): Connection probability filter
    → Top-100 candidates per user
  Stage 2 (CCB2): Rating quality filter
    → Recommend if predicted_rating >= 4.0

Models:
  CCA2: 351,648 parameters
  CCB2: 352,737 parameters

Data:
  Good purchases (rating >= 4.0): 51,830
  Train: 89,294
  Val: 7,480
  Test: 8,365

Validation Performance:
  AUC-ROC: 0.9562
  Accuracy: 0.2136
  Precision: 1.0000
  Recall: 0.2136
  F1 Score: 0.3521

Test Performance:
  Accuracy: 0.2245
  Precision: 1.0000
  Recall: 0.2245
  F1 Score: 0.3667

Ready for comparison with CCA2, CCB2, and other CCC variants!
